# <center>小人脸检测 - Finding Tiny Faces</center>
代码地址:https://github.com/peiyunh/tiny<br>
环境:MATLAB MatConvNet<br>
论文地址:https://arxiv.org/pdf/1612.04402.pdf

![avatar](./tiny_face_img/selfie.png)<br>

### 安装环境

步骤一:<br>
git clone --recursive git@github.com:peiyunh/tiny.git<br>
出现了错误:<br>

    正克隆到 'tiny'...
    Permission denied (publickey).
    fatal: Could not read from remote repository.
    Please make sure you have the correct access rights
    and the repository exists.
解决办法:

    $ ssh-keygen -t rsa -C "1010004295@qq.com"
    出现如下:
    Generating public/private rsa key pair.
    Enter file in which to save the key (/home/hutao/.ssh/id_rsa): 输入:/home/hutao/.ssh/id_rsa_1(要不然覆盖之前的,最好是新建一个路径文件)
    Enter passphrase (empty for no passphrase):(直接按enter)
    Enter same passphrase again:(也是直接enter)
    出现如下:
    Your identification has been saved in /home/hutao/.ssh/id_rsa_1.
    Your public key has been saved in /home/hutao/.ssh/id_rsa_1.pub.
    The key fingerprint is:
    SHA256:XI5bJl+BR33cPuMJwFH+wCdWsL+rCp0eV8VZA69gvY8 1010004295@qq.com
    The key's randomart image is:
    +---[RSA 2048]----+
    |          ..+=o+o|
    |           =+ =.B|
    |          oo+O *o|
    |       . +..ooOoo|
    |        S + .oo++|
    |         *... +o.|
    |        ...+ E o |
    |          o o   .|
    |           o.... |
    +----[SHA256]-----+
    
     打开刚刚生成的key:
     $ cat .ssh/id_rsa_1.pub
     出现如下:
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCbnpYGXF6cAllmlxdXcE4awFuuVbL0iTyfwig7ZdyUX/6zS3Qy6L96EfsLk9g5C5OCWVoxwFMzCfghUJXjmjLnz/qkeoQo1kw9wMJfbWxYNQDwlwZb3TaFW409BdsKse225OnfIKIVw67KoGH2Fh4/kDNTo/k1TT7wpMlE1juvI6AQ2fpw67JnjkPeDRAEdKQb4XYNJPB0hZgGkpWfF9iq0iM3BWB7ZYAVW15/9ruadzFOzUjTkYE5fXBPbWwR3BQq5kcy//+Hh0EQ3Us1Xn1DdFxUaJV28S7vUHJfku/Zj96MPEZeoVwUfMuHs6Cq0hfZx5BA9GkR4+LmQZmJOhBx 1010004295@qq.com
    
    将生成的key复制到自己的github上的SSH keys即可:
![avatar](./tiny_face_img/1.png)<br>         
再次git clone --recursive git@github.com:peiyunh/tiny.git 会出现:

    子模组 'matconvnet' (https://github.com/peiyunh/matconvnet.git) 未对路径 'matconvnet' 注册
    子模组 'toolbox/export_fig' (https://github.com/altmany/export_fig.git) 未对路径 'toolbox/export_fig' 注册
    那是因为递归操作下载,不行的话,按照手动下载到相应的文件夹路径也是一样的;

步骤二:在MATLABcommand下<br>

    >>cd matconvnet/
    >>addpath matlab/
    
    >>vl_compilenn('enableImreadJpeg', true, 'enableGpu', true, 'cudaRoot', [cuda_dir],...
                'cudaMethod', 'nvcc', 'enableCudnn', true, 'cudnnRoot', [cudnn_dir]);
    这里的cuda_dir和cudnn_dir为你要填的路径;我的是:
    >>vl_compilenn('enableImreadJpeg', true, 'enableGpu', true, 'cudaRoot', ['/usr/local/cuda-8.0'],...
                'cudaMethod', 'nvcc', 'enableCudnn', true, 'cudnnRoot', ['/usr/local/cuda-8.0']);
    发现还是出错如下:
        错误使用 vl_compilenn>nvcc_compile (line 536) Command "/usr/local/cuda-8.0/bin/nvcc" -c "/home/hutao/huta/tiny/matconvnet/matlab/src/bits/impl/nnconv_cudnn.cu" -DNDEBUG -DENABLE_GPU
    分析:应该是cuda的问题,或许是版本问题;但是仔细打开vl_compilenn.m文件,发现如果不指定cuda的路径(有可能CUDA路径出错),MATLAB也会有默认的CUDA路径;所以我不需要指定cuda的路径;
    >>vl_compilenn('enableImreadJpeg', true, 'enableGpu', true, ...
                'cudaMethod', 'nvcc', 'enableCudnn', true);
    忽视警告;直至编译成功;
    
    >> vl_testnn('gpu', true);  % vl_testnn('gpu', false) for cpu-only,漫长等待完成如下:
        result = 

      1x3322TestResult 数组(具有属性):

        Name
        Passed
        Failed
        Incomplete
        Duration

    总计:
       3020 Passed, 302 Failed, 302 Incomplete.
       1087.1901 秒测试时间。

步骤三:Compile our MEX function in MATLAB and test if it works as expected:

    >> cd utils/;(而不是matconvnet/utils)
    >> compile_mex;
    >> test_compute_dense_overlap;

步骤四:Download WIDER FACE and unzip data and annotation files to data/widerface such that:
   
    $ ls data/widerface
    wider_face_test.mat   wider_face_train.mat    wider_face_val.mat
    WIDER_test/           WIDER_train/            WIDER_val/



步骤五:Download pretrained model,下载预训练模型:
    
    ResNet101
    ResNet50
    VGG16
    在tiny文件夹下新建一个文件夹:trained_models,将下载好的预训练模型放在复制到trained_models文件夹下;

### Demo

    We provide a minimal demo tiny_face_detector.m that runs our detector on an single input image and output face detections:
    
    function bboxes = tiny_face_detector(image_path, output_path, prob_thresh, nms_thresh, gpu_id)
    比如:
    >> bboxes = tiny_face_detector('data/demo/selfie.jpg', './selfie.png', 0.5, 0.1, 0)


效果如下:<br>
侧脸毫无违和感:
![avatar](./tiny_face_img/5A077C05563232FBE58F427E71290207.png)<br>
几个人明星素颜很赞:
![avatar](./tiny_face_img/20161206054119247.png)<br>
班级开会严肃的样子很直观:
![avatar](./tiny_face_img/DSC_9358.png)<br>
我想说再一次被震撼到!!!
![avatar](./tiny_face_img/timg.png)<br>


### Train训练自己的数据
(尽管作者只有训练自己的人脸检测数据)如果这样效果检测应用到遥感目标舰船检测上去,针对小目标和密集型目标;效果我相信会萌萌哒!!!

#### 一:训练给定的数据集

To train a ResNet101-based Tiny Face Detector, run following command in MATLAB:
    >> hr_res101('train');           % which calls cnn_widerface.m

将gpus = [1 2 3 4]改为%gpus = [1];因为只有一张显卡;<br>
但是显示出错如下:<br>

    train: epoch 01:   1/1074:错误使用 vl_nnconv
        forward: cuDNN error [cudnn: "/home/hutao/hutao/tiny/matconvnet/matlab/src/bits/impl/nnconv_cudnn.cu":141(CUDNN_STATUS_BAD_PARAM)]
        

网上有两个相应的参考链接,但是还是没怎么看懂;<br>
    https://github.com/vlfeat/matconvnet/issues/132<br>
    https://github.com/vlfeat/matconvnet/issues/1022<br>
意思提到不要用cudnn;所以我先编译的时候不选择cudnn编译,改为false,重复之前的步骤如下:<br>

    vl_compilenn('enableImreadJpeg', true, 'enableGpu', true, ...
                'cudaMethod', 'nvcc', 'enableCudnn', false);
                
    1x3322TestResult 数组(具有属性):

    Name
    Passed
    Failed
    Incomplete
    Duration

    总计:
       3322 Passed, 0 Failed, 0 Incomplete.
       1014.1757 秒测试时间。

编译完毕,再执行后面额步骤,然后又开始测试:

    >> bboxes = tiny_face_detector('data/demo/selfie.jpg', './selfie.png', 0.5, 0.1, 0)出错,又得改回:
    >> bboxes = tiny_face_detector('data/demo/selfie.jpg', './selfie.png', 0.5, 0.1, 1)
    才能测试成功.
    
    >> addpath scripts
    >> hr_res101('train');           % which calls cnn_widerface.m
再次出错:

    train: epoch 01:   1/1074:错误使用 gpuArray/max
    Out of memory on device. To view more detail about available memory on the GPU, use 'gpuDevice()'. If the problem persists, reset the GPU
    by calling 'gpuDevice(1)'.
将batch_size改小,从12改为6,(调到8最合适)再次运行;<br>
![avatar](./tiny_face_img/name.jpg)
查看显卡情况:(正在运行,说明训练正常)<br>
<img src="./tiny_face_img/name1.png" align='left'></img><br>  

        训练好的model放在/home/hutao/hutao/tiny/models/widerface-resnet-101-simple-sample256-posfrac0.5-N25-bboxreg-cluster-scaled文件夹下;这样就可以用该模型预测了,效果和原来的model效果差不多. 